In [0]:
# -*- coding: utf-8 -*-
"""
Created on 12-05-2019

@author: Xin Huang
"""

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop


from sklearn.model_selection import train_test_split
import numpy as np

# data = np.load('Sat_data_small.npz')
data = np.load('/content/content/My Drive/Colab Notebooks/Sat_data_small.npz')

# passive = 1


X_v = data['viirs']
Y_v = data['label']
print ('X_v shape:');
print (X_v.shape);

X_c = data['calipso']
Y_c = data['label']
print ('X_c shape:');
print (X_c.shape);

inds_v,vals_v = np.where(Y_v>0)
Y_v = Y_v[inds_v]
X_v = X_v[inds_v]
print ('X_v')
print (X_v)

inds_c,vals_c = np.where(Y_c>0)
Y_c = Y_c[inds_c]
X_c = X_c[inds_c]
print ('X_c')
print (X_c)




Using TensorFlow backend.


FileNotFoundError: ignored

In [0]:
print(X_c[:,0])
print(X_c[:,2])
print(X_c[:,1].shape)

[0.         0.         0.         ... 0.17785511 0.17785511 0.25542167]
[-9999.         -9999.         -9999.         ... -3749.3627354
 -3749.3627354  -4582.63542065]
(341889,)


In [0]:
# combine data and split latter to define ground truth for MLR
from sklearn.linear_model import LinearRegression
n1=20
n2=25
X=np.concatenate((X_v, X_c), axis=1)
Y=Y_v
print (X.shape)
print (Y_v)
x_train, x_temp, y_train, y_temp = train_test_split(X, Y,
                                                    test_size=0.6,
                                                    random_state=0,
                                                    stratify=Y)

x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp,
                                                    test_size=0.5,
                                                    random_state=0,
                                                    stratify=y_temp)

# feature scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
x_train=sc_X.fit_transform(x_train)
x_valid=sc_X.transform(x_valid)
x_test=sc_X.fit_transform(x_test)

x_train_v = x_train[:, 0:20]
x_train_c = x_train[:, 20:45]
print(x_train_v.shape)
print(x_train_c.shape)

x_valid_v = x_valid[:, 0:20]
x_valid_c = x_valid[:, 20:45]
print(x_valid_v.shape)
print(x_valid_c.shape)

x_test_v = x_test[:, 0:20]
x_test_c = x_test[:, 20:45]
print(x_test_v.shape)
print(x_test_c.shape)

(341889, 45)
[[3]
 [3]
 [3]
 ...
 [6]
 [6]
 [6]]
(136755, 20)
(136755, 25)
(102567, 20)
(102567, 25)
(102567, 20)
(102567, 25)


In [0]:
print(x_test_v.shape[0])

102567


In [0]:
# run multivariate linear regression to impute the missing calipso data
mlrs = []
for k in range(0, 25):
    print (k)
    # fit linear model for each kth feature of Calipos train data
    mlr_k = LinearRegression()
    mlr_k.fit(x_train_v, x_train_c[:, k]) 
    print(mlr_k.intercept_)
#     print(mlr_k.coef_)
    mlrs.append(mlr_k)

0
-1.9724703235085413e-09
1
-8.965005798512838e-09
2
3.622831339884026e-09
3
7.398479256587218e-09
4
-6.171363184151272e-09
5
8.697580974148788e-10
6
-1.171193496165701e-09
7
6.51754818068691e-09
8
-9.44268899599902e-10
9
-1.003316953686205e-08
10
1.3778024406751982e-09
11
-9.157746656447646e-09
12
-6.340215421312114e-09
13
-3.091512539149338e-10
14
-7.504688388843246e-09
15
-8.06148547508625e-09
16
-1.0110007630936131e-08
17
6.055412685600499e-09
18
-1.7208838494291117e-09
19
6.014604733082582e-09
20
-7.175106776938481e-09
21
-5.106170397611972e-09
22
-4.253749142161028e-10
23
-5.914117093119537e-10
24
-8.043567905126904e-09


In [0]:
# define the x_test_pt used for testing
x_test_c_p = np.zeros(shape=(25, x_test_v.shape[0]))
print(x_test_c_p.shape)
# predict the kth feature of the Calipso data from the Viris test data, Assume the Calipso test data is not existed. 
for k in range(0, 25):
    predi = mlrs[k].predict(x_test_v)
#     print(predi)
    x_test_c_p[k] = predi
    
print(x_test_c_p.shape)
# print(x_test_c_p)

x_test_c_pt = x_test_c_p.T
print(x_test_v.shape)
print(x_test_c_pt.shape)
# print(x_test_c_pt)
x_test_pt = np.concatenate((x_test_v, x_test_c_pt),axis=1)
print(x_test_pt.shape)

(25, 102567)
(25, 102567)
(102567, 20)
(102567, 25)
(102567, 45)


In [0]:
#defien the x_train_pt used for DL training
x_train_c_p = np.zeros(shape=(25, x_train_v.shape[0]))
print(x_train_c_p.shape)
# predict the kth feature of the Calipso data from the Viris test data, Assume the Calipso test data is not existed. 
for k in range(0, 25):
    predi = mlrs[k].predict(x_train_v)
#     print(predi)
    x_train_c_p[k] = predi
    
print(x_train_c_p.shape)

x_train_c_pt = x_train_c_p.T
print(x_train_v.shape)
print(x_train_c_pt.shape)

x_train_pt = np.concatenate((x_train_v, x_train_c_pt),axis=1)
print(x_train_pt.shape)

(25, 136755)
(25, 136755)
(136755, 20)
(136755, 25)
(136755, 45)


In [0]:
#defien the x_valid_pt used for DL training as valid dataset
x_valid_c_p = np.zeros(shape=(25, x_valid_v.shape[0]))
print(x_valid_c_p.shape)
# predict the kth feature of the Calipso data from the Viris test data, Assume the Calipso test data is not existed. 
for k in range(0, 25):
    predi = mlrs[k].predict(x_valid_v)
#     print(predi)
    x_valid_c_p[k] = predi
    
print(x_valid_c_p.shape)

x_valid_c_pt = x_valid_c_p.T
print(x_valid_v.shape)
print(x_valid_c_pt.shape)

x_valid_pt = np.concatenate((x_valid_v, x_valid_c_pt),axis=1)
print(x_valid_pt.shape)

(25, 102567)
(25, 102567)
(102567, 20)
(102567, 25)
(102567, 45)


In [0]:
# use Logistic regression model
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial').fit(x_train_pt, y_train)
y_pred = clf.predict(x_test_pt)
print (clf.score(x_test_pt, y_test))

num_classes1 = 6
y_lr_gd = keras.utils.to_categorical(y_test-1, num_classes1)
y_lr_pred = keras.utils.to_categorical(y_pred-1, num_classes1)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_lr_gd, y_lr_pred)

/Users/xin/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5457213333723322


/Users/xin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.700089749720584

In [0]:
# use deep learning model
num_classes = 6

print ('y_train')
print (y_train)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train-1, num_classes)
y_valid = keras.utils.to_categorical(y_valid-1, num_classes)
y_test = keras.utils.to_categorical(y_test-1, num_classes)

print ('y_train converted')
print (y_train)

batch_size = 256
epochs = 30

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x_train_pt.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

history = model.fit(x_train_pt, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valid_pt, y_valid))
score = model.evaluate(x_test_pt, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test score:', score)

y_train
[[6]
 [6]
 [2]
 ...
 [2]
 [2]
 [6]]
y_train converted
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_133 (Dense)            (None, 512)               23552     
_________________________________________________________________
dropout_111 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_134 (Dense)            (None, 256)               131328    
_________________________________________________________________
dropout_112 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_135 (Dense)            (None, 128)               32896     
_________________________________________________________________
dropout_113 (D

In [0]:
score = model.evaluate(x_test_pt, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test score:', score)
predict_result = model.predict_proba(x_test_pt);
print (predict_result)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predict_result)

Test loss: 0.8377433909601574
Test accuracy: 0.690621739937009
Test score: [0.8377433909601574, 0.690621739937009]
[[5.7129534e-03 5.3762001e-01 5.6491017e-02 1.7452382e-01 5.0240867e-03
  2.2062805e-01]
 [6.5439389e-07 1.0392166e-03 8.1740141e-01 6.0047988e-02 3.4940385e-06
  1.2150720e-01]
 [6.3388225e-11 2.2870819e-10 2.9165699e-14 1.4521619e-17 9.9723804e-01
  2.7618951e-03]
 ...
 [5.1664990e-01 6.7473732e-02 1.4397442e-01 2.5002649e-02 1.5300347e-01
  9.3895823e-02]
 [5.6942576e-01 2.0718051e-05 2.8022971e-02 2.7882309e-06 3.8649851e-01
  1.6029309e-02]
 [2.2915234e-08 3.0617854e-12 6.0225457e-12 2.8930152e-18 9.9916029e-01
  8.3967426e-04]]


0.9160004305804584

In [0]:
# # Experiments, references 

# # use deep learning model
# batch_size = 256
# epochs = 15

# model = Sequential()
# model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
# model.add(Dropout(0.5))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))

# model.summary()

# model.compile(loss='categorical_crossentropy',
#               optimizer=RMSprop(lr=0.001),
#               metrics=['accuracy'])

# history = model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_valid, y_valid))
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
# print('Test score:', score)


In [0]:
# predict_result = model.predict_proba(x_test);
# print (predict_result)
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_test, predict_result)

In [0]:
# # references

# from sklearn.linear_model import LinearRegression
# mlr = LinearRegression()
# # Fit linear regression.
# mlr.fit(X_v, X_c[:,0])
# # Get the slope and intercept of the line best fit.
# print(mlr.intercept_)
# print(mlr.coef_)